In [2]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
from gtts import gTTS
# for language model
import transformers
import os
import time
# for data
import os
import datetime
import numpy as np

/run/media/vismay/New Volume/sem-6/NLP/speech/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-14 16:41:54.963369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-14 16:41:54.963502: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-14 16:42:07.771775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-14 16:42:07.772587: W tensorflow/compiler/xla/stream_ex

In [3]:
class ChatBot():
  def __init__(self,name):
    print("------get ready to meet",name,"------")
    self.name = name
  def speech_to_text(self):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting noise ")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Listening....")
        recorded_audio = recognizer.listen(source, timeout=4)
        # print("Done recording")
        self.text = "ERROR"

    try:
        # print("Recognizing the text")
        self.text = recognizer.recognize_google(
                recorded_audio, 
                language="en-US"
            )
        print("me -->: {}".format(self.text))
    except Exception as ex:
        print(ex)
  @staticmethod
  def text_to_speech(text):
    print("AI-->",text)
    speaker = gTTS(text = text , lang = "en" , slow = False)
    speaker.save("res.mp3")
    statbuf = os.stat("res.mp3")
    mbytes = statbuf.st_size / 1024
    duration = mbytes / 200
    os.system("mpg123 res.mp3")
    time.sleep(int(50*duration))
    os.remove("res.mp3")
  def wake_up(self, text):
    return True if self.name in text.lower() else False
  @staticmethod
  def action_time():
    return datetime.datetime.now().time().strftime('%H:%M')

In [4]:
if __name__ == "__main__":
  ai = ChatBot(name = 'jarvis')
  nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
  # nlp(transformers.Conversation(ai.text), pad_token_id=50256)
  os.environ["TOKENIZERS_PARALLELISM"] = "true"
  ex=True
  while ex:
    ai.speech_to_text()
    # wake up to call
    if ai.wake_up(ai.text) is True:
      res = "Hello i am jarvis the AI , what can i do for you?"
      ai.text_to_speech(res)
    # any action as time
    elif "time" in ai.text:
      res = ai.action_time()
    # polite response to thanks
    elif any(i in ai.text for i in ['thanks','thank','thankyou']):
      res = np.random.choice(["you're welcome!","anytime","no worries","cool","bas kya yaar!"])
    elif any(i in ai.text for i in ["exit","close","bye"]):
      res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","see you soon!"])
      ex = False
    # convo
    else:
      if ai.text == "ERROR":
        res = "Sorry , come again!"
      else:
        chat = nlp(transformers.Conversation(ai.text) , pad_token_id = 50256)
        res = str(chat)
        res = res[res.find("bot >>")+6:].strip()
    ai.text_to_speech(res)
  print("-----Jarvis shutting down-----")


------get ready to meet jarvis ------


Downloading: 100%|██████████| 642/642 [00:00<00:00, 132kB/s]
Downloading:   0%|          | 3.44M/1.42G [00:09<1:27:34, 270kB/s] 

KeyboardInterrupt: 

Downloading:   0%|          | 3.46M/1.42G [00:26<1:27:34, 270kB/s]